#### <STRONG>Lapage - Partie 2

## Table des matières<br>
    
* [Analyse du comportement des clients](#Analyse_du_comportement_des_clients)
    * [Répartition du CA par client](#Répartition_du_CA_par_client)
    * [Corrélation entre genre et catégories](#Corrélation_entre_genre)
    * [Corrélation entre âge et achat](#Corrélation_entre_âge_et_achat)
        * [Âge et montant des achats](#Âge_et_montant_des_achats)
        * [Âge et fréquence d'achats](#Age_et_fréquence_d_achats)
        * [Âge et panier moyen](#Âge_et_panier_moyen)
            * [Âge et panier moyen](#Âge_et_panier_moyen)
            * [Tranche d'âge et panier moyen](#Tranche_age_et_panier_moyen)
        * [Âge et catégories de produits achetés](#Âge_et_catégories_de_produits_achetés)

#### <strong>Mise en place de l'environnement

In [ ]:
# import des librairies
import numpy as np
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as st
import plotly.figure_factory as ff

In [ ]:
# import des .csv
lapage_bdd = pd.read_csv('lapage_bdd.csv',parse_dates=['birth','date'])

# Convertion de la colonne 'categ' en 'object'
lapage_bdd['categ'] = lapage_bdd['categ'].astype(str)

------------
<li style="font-size: 20px;"><strong>Analyse du comportement <span style="color: rgb(184, 49, 47);">des clients</span></strong></li>
</div></span></p><a class="anchor" id="Analyse_du_comportement_des_clients"></a>

------------

<li style="font-size: 18px;"><strong>5.1 - Répartition du CA par client </strong></li>
</div></span></p><a class="anchor" id="Répartition_du_CA_par_client"></a>

##### <strong>Préparation du DataFrame

In [ ]:
# Création DataFrame
repart_ca_clients = lapage_bdd.groupby(by=["client_id"]).sum().reset_index()

# sélection des colonnes à afficher
repart_ca_clients = repart_ca_clients[['client_id','price']]

# Affichage DataFrame
repart_ca_clients.head()

##### <strong>Recherche de valeurs aberrantes par Zscore

In [ ]:
# Calcul du "zscore"
repart_ca_clients['zscore'] = ((repart_ca_clients['price'] - repart_ca_clients['price'].mean())/
                               repart_ca_clients['price'].std())

# Création colonne "zscore_outlier"
repart_ca_clients["zscore_outliers"] = (abs(repart_ca_clients['zscore'])>1.96)

# Affichage des résultats
print("Nombre de valeurs non aberrantes: " + str(repart_ca_clients.zscore_outliers.value_counts()[0]))
print("Nombre de valeurs aberrantes: " + str(repart_ca_clients.zscore_outliers.value_counts()[1]))

In [ ]:
# Création DataFrame (fichier des valeurs aberrantes)
repart_ca_clients_outliers = repart_ca_clients[abs(
    repart_ca_clients['zscore'] > 1.96)].sort_values(by=['price'],
                                                     ascending = False)

# sélection des colonnes à afficher
repart_ca_clients_outliers = repart_ca_clients_outliers[['client_id','price','zscore_outliers']]

# Affichage DataFrame
repart_ca_clients_outliers

In [ ]:
# Création DataFrame (fichier sans valeurs aberrantes)
repart_ca_clients_no_outliers = repart_ca_clients[abs(
    repart_ca_clients['zscore'] < 1.96)].sort_values(by=['price'],
                                                     ascending = False)

# sélection des colonnes à afficher
repart_ca_clientsno_outliers = repart_ca_clients_no_outliers[['client_id','price','zscore_outliers']]

# Affichage DataFrame
repart_ca_clients_no_outliers.head()

In [ ]:
# Calcul de la représentation du CA des outliers
# Groupements par catégories de outliers
repart_pop_outliers = (repart_ca_clients.groupby(by=["zscore_outliers"],
                                             dropna=False).count()).reset_index()

# Calcul du % de CA
repart_pop_outliers['% pop'] = (((repart_pop_outliers['client_id']) /
                               (repart_pop_outliers['client_id']).sum()) * 100).round(4)

# Suprression de la colonne
repart_pop_outliers = repart_pop_outliers.drop(columns=['price','zscore'])

# Affichage DataFrame
repart_pop_outliers

In [ ]:
# Calcul de la représentation du CA des outliers
# Groupements par catégories de outliers
repart_ca_outliers = (repart_ca_clients.groupby(by=["zscore_outliers"],
                                             dropna=False).sum()).reset_index()

# Calcul du % de CA
repart_ca_outliers['% CA'] = (((repart_ca_outliers['price']) /
                               (repart_ca_outliers['price']).sum()) * 100).round(4)

# Suprression de la colonne
repart_ca_outliers = repart_ca_outliers.drop(columns=['zscore'])
    
# Affichage DataFrame
repart_ca_outliers

<div class="alert alert-block alert-info">
<b>Note :</b> On constate lors du nettoyage des données que 4 clients représentent 7.43% du CA pour seulement 0.046% du nombre de clients total.
</div>

##### <strong>Courbe de Lorenz

In [ ]:
# Courbe de Lorenz
repart_ca_clients = repart_ca_clients[repart_ca_clients['price'] > 0]
dep = repart_ca_clients['price'].values
n = len(dep)
lorenz = np.cumsum(np.sort(dep)) / dep.sum()
lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0
medial = lorenz[(lorenz < 0.5)].size / lorenz.size
xaxis = np.linspace(0-1/n,1+1/n,n+1) 

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1],
                    mode='lines', name='Equality', line=dict(color="red")))

fig.add_trace(go.Scatter(x=xaxis, y=lorenz,
                    mode='lines', name='Lorenz', line=dict(color="cyan")))

fig.add_trace(go.Scatter(x=[0,medial], y=[0.5,0.5],
                    mode='lines', name='Mediale CA', line=dict(color="royalblue")))

fig.add_trace(go.Scatter(x=[medial,medial], y=[0,0.5],
                    mode='lines', name='Mediale client', line=dict(color="darkblue")))

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', 
    font_color="black", autosize=False, width=600, height=600, showlegend=True,
    title=dict(text='<b>Répartition CA par client</b>'),
    xaxis=dict(title='Part clients cumulée (%)'), 
    yaxis=dict(title='Chiffre d\'affaires cumulée (%)'),
    legend=dict(orientation="h", y=1.05, xanchor="right", x=1))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Indice de Gini

In [ ]:
# Calcul indice de Gini (avec outliers)
# Surface sous la courbe de Lorenz
AUC = (lorenz.sum() - lorenz[-1]/2 - lorenz[0]/2)/n
# surface entre la première bissectrice et le courbe de Lorenz
S = 0.5 - AUC 
gini = 2*S
print('Indice de Gini:',gini.round(2))

<div class="alert alert-block alert-info">
<b>Note :</b> L'indice de Gini confirme une répartition du CA assez équilibrée.<br>
En revanche, 4 clients représentent une part de CA supérieur à 7%, ce qui peut faire penser à des clients de type professionnels.
</div>

<li style="font-size: 18px;"><strong>5.2 - Corrélation entre genre et catégories des livres achetés </strong></li>
</div></span></p><a class="anchor" id="Corrélation_entre_genre_et_catégories_des_livres_achetés"></a>

##### <strong> Comparaison entre 2 variables qualitatives

##### <strong> Test du Chi-2

In [ ]:
# Création de la matrice "valeurs observées" (tableau de contingence)
X = "sex"
Y = "categ"
cont = lapage_bdd[[X, Y]].pivot_table(index=X, columns=Y, aggfunc=len).fillna(0).copy()
tx = lapage_bdd[X].value_counts()
ty = lapage_bdd[Y].value_counts()
cont = cont.astype(int)

# Affichage tableau
cont

In [ ]:
# Création de la matrice "valeurs attendues"
tx_df = pd.DataFrame(tx)
tx_df.columns = ["c"]
ty_df = pd.DataFrame(ty)
ty_df.columns = ["c"]

# Valeurs totales observées
n = len(lapage_bdd)

# Produit matriciel. On utilise pd.T pour pivoter une des deux séries.
indep = (tx_df.dot(ty_df.T) / n).round(2)

# Affichage tableau
indep

In [ ]:
# Calcul de la matrice "écart au carré normalisé de la valeur attendue VS valeur observée"
freq = ((cont-indep)**2/indep).round(2)

# Affichage tableau
freq

In [ ]:
# Calcul du Chi2
chi2 = (freq.sum().sum()).round(2)

# Affichage résultat
chi2

##### <strong> Test du Chi-2 via scipy.stats

In [ ]:
# test du Chi2
st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)

In [ ]:
# chi2, p, dof, ex = stats.chi2_contingency(categ_sex)
print ('Chi2:',st_chi2.round(2))
print('Degré de liberté:', st_dof)
print('P-Value:', st_p)
print('Table théorique:')
pd.DataFrame(st_exp.round(2))

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'sex'</b> et <b>'categ'</b> sont indépendantes.<br><br>
Avec un risque de se tromper (rejeter à tort l'hypothèse nulle) égal à 5 % pour un d.d.l. de 2, la valeur critique trouvée dans les tables serait de 5,991 or notre Chi2 est de 147,12.<br><br>    
On peut donc rejeter H0 et conclure qu'il y a une corrélation entre <b>'sex'</b> et <b>'categ'</b>.
</div>

##### <strong> Heatmap

In [ ]:
# Création de la matrice
cont_heat = lapage_bdd[[X, Y]].pivot_table(index=X, columns=Y, aggfunc=len).fillna(0).copy()
nmp = cont_heat.to_numpy()
z = nmp

x = ['Categ 1', 'Categ 2', 'Categ 3']
y = ['Femmes', 'Hommes']

# Création du tracé
fig = ff.create_annotated_heatmap(z, x=x, y=y,colorscale=[[0, 'cyan'],
                                              [1.0, 'royalblue']])

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre genre et catégorie</b>', font_size=18))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='rgba(0,0,0,0)')
fig.update_yaxes(gridcolor='rgba(0,0,0,0)', linecolor='rgba(0,0,0,0)')

# Affichage graphique
fig.show()

<li style="font-size: 18px;"><strong>5.3 - Corrélation entre âge et achat </strong></li>
</div></span></p><a class="anchor" id="Corrélation_entre_âge_et_achat"></a>

<div class="alert alert-block alert-info">
<b>Note :</b> Pour cette partie, nous utiliserons la base de donnée sans les 4 gros clients. Ces derniers pourraient biaiser les résultats tant par leur représentation numérique mais surtout par leur représentation en terme de CA.
</div>

##### <strong>Préparation du DataFrame

In [ ]:
#Fichier sans les 4 gros clients
lapage_bdd_NO = lapage_bdd.loc[(lapage_bdd['client_id']!='c_1609')]
lapage_bdd_NO = lapage_bdd_NO.loc[(lapage_bdd_NO['client_id']!='c_4958')]
lapage_bdd_NO = lapage_bdd_NO.loc[(lapage_bdd_NO['client_id']!='c_6714')]
lapage_bdd_NO = lapage_bdd_NO.loc[(lapage_bdd_NO['client_id']!='c_3454')]

# Tri croissant par catégorie d'âge
lapage_bdd_NO = lapage_bdd_NO.sort_values(by='age_range', ascending=True)

# Affichage DataFrame
lapage_bdd_NO.head()

<li style="font-size: 16px;"><strong>5.3.1 - Âge et montant des achats </strong></li>
</div></span></p><a class="anchor" id="Âge_et_montant_des_achats"></a>

##### <strong> Comparaison entre 2 variables quantitatives

In [ ]:
# Création du tracé
fig = px.scatter(lapage_bdd_NO, x='age', y='price',
            color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et montant des achats</b>', font_size=18), 
    xaxis=dict(title='Âge'), yaxis=dict(title='Montant des achats en (€)'))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Pearson

In [ ]:
# Calcul du coefficient de corrélation
r, p = st.pearsonr(lapage_bdd_NO['age'],lapage_bdd_NO['price'])
print('Pearson: ',r)
print('P-Value: ',p)

# Matrice de covariance
print('Covariance: ',np.cov(lapage_bdd_NO['age'],lapage_bdd_NO['price'],ddof=0)[1,0])

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'price'</b> sont indépendantes.<br><br>
Le test de Pearson étant proche de 0, on peut donc confirmer H0 et conclure qu'il n'y a pas corrélation entre entre <b>'age'</b> et <b>'price'</b>.
</div>

##### <strong> Comparaison entre 2 variables quantitatives et qualititatives

In [ ]:
# Création du tracé
fig = px.box(lapage_bdd_NO, x='price', y='age_range',
            color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et montant des achats</b>', font_size=18), 
    xaxis=dict(title='Montant des achats en (€)'), yaxis=dict(title='Tranche d\'âge'))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Anova

In [ ]:
# Calcul eta squared
X = lapage_bdd_NO['age_range'] # qualitative
Y = lapage_bdd_NO['price'] # quantitative

def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
eta_squared(X,Y)

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'price'</b> sont indépendantes.<br><br>
Le test Anova étant proche de 0, on peut donc confirmer H0 et conclure qu'il n'y a pas corrélation entre entre <b>'age'</b> et <b>'price'</b>.
</div>    

<li style="font-size: 16px;"><strong>5.3.2 - Âge et fréquence d'achats</strong></li>
</div></span></p><a class="anchor" id="Age_et_fréquence_d_achats"></a>

##### <strong> Comparaison entre 2 variables quantitatives

In [ ]:
# Création DataFrame
age_freq = lapage_bdd_NO.groupby(by=['client_id','age','age_range']).size().reset_index(name='freq').sort_values(by='age')

# Affichage DataFrame
age_freq.head()

In [ ]:
# Création du tracé
fig = px.scatter(age_freq, x='age', y='freq',
                 color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', 
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et fréquence d\'achats</b>', font_size=18),
    xaxis=dict(title='Âge'), yaxis=dict(title='Fréquence d\'achats en (€)'))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Pearson

In [ ]:
# Calcul du coefficient de corrélation
r, p = st.pearsonr(age_freq['age'],age_freq['freq'])
print('Pearson: ',r)
print('P-Value: ',p)

# Matrice de covariance
print('Covariance: ',np.cov(age_freq['age'],age_freq['freq'],ddof=0)[1,0])

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'freq'</b> sont indépendantes.<br><br>
Le test de Pearson étant proche de 0, on peut donc confirmer H0 et conclure qu'il n'y a pas corrélation entre entre <b>'age'</b> et <b>'freq'</b>.
</div>

##### <strong> Comparaison entre 2 variables quantitatives et qualitatives

In [ ]:
# Création du tracé
fig = px.box(age_freq, x='freq', y='age_range',
                 color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', 
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et fréquence d\'achats</b>', font_size=18),
    xaxis=dict(title='Âge'), yaxis=dict(title='Fréquence d\'achats en (€)'))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Anova

In [ ]:
# Calcul eta squared
X = age_freq['age_range'] # qualitative
Y = age_freq['freq'] # quantitative

def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
eta_squared(X,Y)

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'freq'</b> sont indépendantes.<br><br>
Le test Anova étant proche de 0, on peut donc confirmer H0 et conclure qu'il n'y a pas corrélation entre entre <b>'age'</b> et <b>'freq'</b>.
</div>

<li style="font-size: 16px;"><strong>5.3.3 - Âge et panier moyen </strong></li>
</div></span></p><a class="anchor" id="Âge_et_panier_moyen"></a>

##### <strong> Comparaison entre 2 variables quantitatives

In [ ]:
# Création DataFrame
corr_age_panier = lapage_bdd_NO.groupby(by=['client_id','age','age_range']).mean().reset_index().sort_values(by='age_range')

# Affichage DataFrame
corr_age_panier.head()

In [ ]:
# Création du tracé
fig = px.scatter(corr_age_panier, x='age', y='price',
             color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et panier moyen</b>', font_size=18),
    xaxis=dict(title='Âge'), yaxis=dict(title='Panier moyen en (€)'),
    margin=dict(  l=50,  r=50, b=50, t=50,pad=4))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Pearson

In [ ]:
# Calcul du coefficient de corrélation
r, p = st.pearsonr(corr_age_panier['age'],corr_age_panier['price'])
print('Pearson: ',r)
print('P-Value: ',p)

# Matrice de covariance
print('Covariance: ',np.cov(corr_age_panier['age'],corr_age_panier['price'],ddof=0)[1,0])

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'price'</b> sont indépendantes.<br><br>
Le test de Pearson étant de 0.51, on peut ni confirmer ni infirmer H0, en revanche on note un certain rapport de cause à effet entre <b>'age'</b> et <b>'price'</b> en fonction du regroupement de certaines tranches d'âge.
</div>

##### <strong> Comparaison entre 2 variables quantitatives et  qualitatives

In [ ]:
# Création du tracé
fig = px.box(corr_age_panier, x='price', y='age_range',
             color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre tranche d\'âge et panier moyen</b>', font_size=18),
    xaxis=dict(title='Panier moyen en (€)'), yaxis=dict(title='Âge'),
    margin=dict(  l=50,  r=50, b=50, t=50,pad=4))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Anova

In [ ]:
# Calcul eta squared
X = corr_age_panier['age_range'] # qualitative
Y = corr_age_panier['price'] # quantitative

def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
eta_squared(X,Y)

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'X'</b> et <b>'Y'</b> sont indépendantes.<br><br>
Le test de Anova étant de 0.57, on peut ni confirmer ni infirmer H0, en revanche on note un certain rapport de cause à effet entre <b>'age'</b> et <b>'price'</b> en fonction du regroupement de certaines tranches d'âge.

<li style="font-size: 16px;"><strong>5.3.4 - Âge et catégories de produits achetés </strong></li>
</div></span></p><a class="anchor" id="Âge_et_catégories_de_produits_achetés"></a>

##### <strong> Comparaison entre 2 variables quantitatives et  qualitatives

In [ ]:
# Création du tracé
fig = px.box(lapage_bdd_NO, x='age', y="categ", 
             color_discrete_sequence=px.colors.qualitative.Pastel)

# Paramètres du graphique
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', 
    font_color="black", width=850, height=400,
    title=dict(text='<b>Corrélation entre âge et catégorie</b>', font_size=18),
    xaxis=dict(title='Âge'), yaxis=dict(title='Catégorie'))

fig.update_xaxes(gridcolor='rgba(0,0,0,0)', linecolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey', linecolor='lightgrey')

# Affichage graphique
fig.show()

##### <strong>Anova

In [ ]:
# Calcul eta squared
X = lapage_bdd_NO['categ'] # qualitative
Y = lapage_bdd_NO['age'] # quantitative

def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
eta_squared(X,Y)

<div class="alert alert-block alert-warning">
<b>Remarques :</b><br><br>
 L'hypothèse nulle (H0) de ce test est la suivante : les deux variables <b>'age'</b> et <b>'categ'</b> sont indépendantes.<br><br>
Le test Anova étant proche de 0, on peut donc confirmer H0 et conclure qu'il n'y a pas de corrélation entre <b>'age'</b> et <b>'categ'</b>.
</div>

#### <STRONG>Fin du notebook